In [1]:
%config Completer.use_jedi = False

## Daily Aggregrage
- dictionary is done
- aggreage both domain, tag and tag_gp

### 0. Initialization

In [2]:
import cx_Oracle
import pandas as pd
from smtutil import common, hiveutil
from datetime import datetime
import numpy as np
import csv, pickle

import os
import argparse
import shutil
import logging
import sys
import yaml
import pyspark.sql.functions as F

In [3]:
logger = logging.getLogger(__name__)
log_format = "[%(asctime)s][%(funcName)25s:%(lineno)4d] %(message)s"
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format=log_format)

In [4]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [5]:
# def main(ssl_data_dir, t_domain_base_dir, spark=None, num_of_dates_to_load=4):

#     # define sc for checking to closing spark
#     sc=False

#     # get spark
#     if not spark:
#         sc, spark = hiveutil.get_sc_spark()
#         logging.info("No spark session. Created one now")

#     #ssl_data_dir = '/app/sna_bdmrdev01/ws-ic/hive/prd_biz_summ_vw.db/vw_idp_ssl_sed_call_summ/'
#     #t_domain_base_dir = '/app/sna_bdmrdev01/ws-ic/hive/weblog.db/'
#     #num_of_dates_to_load = 4

#     ## ---------------------------- Path Settings ------------------------------ ##
#     t_domain_daily_dir = t_domain_base_dir + '/t_domain_daily/'
#     t_domain_index_dir = t_domain_base_dir + '/t_domain_index/'

#     ## ---------------------------- Get Sdfs ------------------------------ ##
#     sdf_ssl = spark.read.parquet(ssl_data_dir)
#     sdf_tdd = spark.read.parquet(t_domain_daily_dir)
#     sdf_tdi = spark.read.parquet(t_domain_index_dir)

#     ## ------------------------ Get Dates to Load ------------------------ ##
#     df_loaded = sdf_tdd.select('date_id').distinct().toPandas()
#     df_legal = sdf_ssl.select('date_id').distinct().filter(F.col('date_id')>=20200101).toPandas()

#     # get list of dates to load
#     dates_to_loaded = df_legal[~df_legal['date_id'].isin(df_loaded['date_id'])].sort_values(by='date_id', ascending=False)['date_id'].tolist()[:num_of_dates_to_load]
#     logging.info(f'Dates to loaded: {dates_to_loaded}')

#     ## --------------------- Aggregate Summary ----------------------  ##
#     df_tdd_count = pd.DataFrame([x.split('=')[1][:6] for x in os.listdir(t_domain_daily_dir) if '=' in x], columns=['month_id'])
#     logging.info(df_tdd_count['month_id'].value_counts().sort_index())

#     ## ------------------------ Aggregate Domain ---------------------- ##
#     for date_id in dates_to_loaded: # 20 mins
#         logging.info(f'date_id: {date_id}, datetime.now: {datetime.now()}')
#         sdf_tdd_new = sdf_ssl.filter(F.col('date_id')==date_id) \
#                          .join(F.broadcast(sdf_tdi), sdf_ssl.ssl_cert_domain == sdf_tdi.domain, how='left') \
#                          .select(
#                             'subr_num',
#                             F.col('domain_index').alias('agg_value'),
#                             F.lit('domain').alias('agg_level'),
#                             F.lit(0).alias('in_byte'),
#                             F.lit(0).alias('out_byte'),
#                             'byte_count',
#                             'total_hit',
#                             F.lit(0).alias('duration'),
#                             'date_id',
#                         )

#         output_dir = t_domain_daily_dir + f'/date_id={date_id}' 
#         try:
#             shutil.rmtree(output_dir)
#             logging.info(f'Removed Data {output_dir}')
#         except FileNotFoundError:
#             pass

#         # write
#         sdf_tdd_new.coalesce(10) \
#                    .write.mode('append') \
#                    .partitionBy('date_id') \
#                    .parquet(t_domain_daily_dir)

#     # stop spark session and spark content
#     if sc == False:
#         spark.stop()
#         sc.stop()
#         logging.info("Stopped spark session")

#     logging.info(f"End, {datetime.now()}")

In [6]:
sc, spark = hiveutil.get_sc_spark()
hiveutil.register_common_udf(spark)

In [7]:
ssl_data_dir = '/app/sna_bdmrdev01/ws-ic/hive/prd_biz_summ_vw.db/vw_idp_ssl_sed_call_summ/'
t_domain_base_dir = '/app/sna_bdmrdev01/ws-ic/hive/weblog.db/'

# # debug
# ssl_data_dir = '/app/ws-ic/hive/prd_biz_summ_vw.db/vw_idp_ssl_sed_call_summ/'
# t_domain_base_dir = '/app/ws-ic/hive/weblog.db/'

num_of_dates_to_load = 4

## path settings
t_domain_daily_dir = t_domain_base_dir + '/t_domain_daily/'
t_domain_index_dir = t_domain_base_dir + '/t_domain_index/'
# t_domain_index_backup_dir = t_domain_base_dir + '/t_domain_index_backup/'
# t_domain_index_new_dir = t_domain_base_dir + '/t_domain_index_new/'

In [8]:
# list of data to loaded
sdf_ssl = spark.read.parquet(ssl_data_dir)
sdf_tdd = spark.read.parquet(t_domain_daily_dir)
sdf_tdi = spark.read.parquet(t_domain_index_dir)

df_loaded = sdf_tdd.select('date_id').distinct().toPandas()
df_legal = sdf_ssl.select('date_id').distinct().filter(F.col('date_id')>=20200101).toPandas()

In [9]:
dates_to_loaded = df_legal[~df_legal['date_id'].isin(df_loaded['date_id'])].sort_values(by='date_id', ascending=False)['date_id'].tolist()[:num_of_dates_to_load]
logging.info(f'dates to loaded: {dates_to_loaded}')

[2023-02-07 11:41:57,951][                 <module>:   2] dates to loaded: [20230206, 20230205, 20230204, 20230203]


### 1. Aggregation

In [10]:
sdf_tdd_summary = spark.read.parquet(t_domain_daily_dir).dropDuplicates(['date_id']).withColumn('date_id', F.col('date_id').cast('string'))

In [11]:
df_tdd_count = pd.DataFrame([x.split('=')[1][:6] for x in os.listdir(t_domain_daily_dir) if '=' in x], columns=['month_id'])
print(df_tdd_count['month_id'].value_counts().sort_index())

202206     5
202207    31
202208    10
202209    30
202210    31
202211    30
202212    31
202301    15
Name: month_id, dtype: int64


#### 1.1. Aggregate Domain

In [12]:
dates_to_loaded

[20230205, 20230204, 20230203, 20230202]

In [ ]:
# dates_to_loaded = df_legal[~df_legal['date_id'].isin(df_loaded['date_id'])].sort_values(by='date_id', ascending=False)['date_id'].tolist()[:22]
# dates_to_loaded

In [67]:
# sdf_tdd_new = sdf_ssl.filter(F.col('date_id')==date_id).cache()
# .join(F.broadcast(sdf_tdi), sdf_ssl.ssl_cert_domain == sdf_tdi.domain, how='left')
# sdf_tdd_new.limit(5).show()

In [14]:
for date_id in dates_to_loaded: # 20 mins
    print(date_id, datetime.now())
    sdf_tdd_new = sdf_ssl.filter(F.col('date_id')==date_id) \
                     .join(sdf_tdi, sdf_ssl.ssl_cert_domain == sdf_tdi.domain, how='left') \
                     .select(
                        'subr_num',
                        F.col('domain_index').alias('agg_value'),
                        F.lit('domain').alias('agg_level'),
                        F.lit(0).alias('in_byte'),
                        F.lit(0).alias('out_byte'),
                        'byte_count',
                        'total_hit',
                        F.lit(0).alias('duration'),
                        'date_id',
                    )

    output_dir = t_domain_daily_dir + f'/date_id={date_id}' 
    try:
        shutil.rmtree(output_dir)
        print(f'[Info] Removed {output_dir}')
    except FileNotFoundError:
        pass

    # write
    sdf_tdd_new.coalesce(10) \
               .write.mode('append') \
               .partitionBy('date_id') \
               .parquet(t_domain_daily_dir)

print(datetime.now())

20230206 2023-02-07 11:42:25.170158
20230205 2023-02-07 11:44:42.023633
20230204 2023-02-07 11:46:29.685681
20230203 2023-02-07 11:48:22.819279
20230202 2023-02-07 11:50:31.923384
20230201 2023-02-07 11:52:44.780000
20230131 2023-02-07 11:55:00.315445
20230130 2023-02-07 11:57:14.819524
20230129 2023-02-07 11:59:37.503790
20230128 2023-02-07 12:01:51.674107
20230127 2023-02-07 12:04:16.384796
20230126 2023-02-07 12:07:10.714216
20230125 2023-02-07 12:09:48.264375
20230124 2023-02-07 12:12:06.924729
20230123 2023-02-07 12:14:13.173283
20230122 2023-02-07 12:16:20.283706
20230121 2023-02-07 12:18:23.812583
20230120 2023-02-07 12:20:42.397185
20230119 2023-02-07 12:23:11.988905
20230118 2023-02-07 12:25:42.826976
20230117 2023-02-07 12:28:14.963462
20230116 2023-02-07 12:30:48.004205
2023-02-07 12:33:23.946485


In [15]:
spark.stop()
sc.stop()